### Imports

In [50]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [51]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [52]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [53]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df[["rssi", "etx", "delay", "busy_fraction"]].values

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [54]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [55]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-60    1.3   5      0.45   1        Bom      0.98      
-67    1.7   3      0.26   1        Bom      0.99      
-74    4.1   19     0.52   0        Ruim     0.95      
-64    1.3   15     0.50   1        Bom      0.93      
-60    3.4   70     0.78   0        Ruim     0.99      
-70    6.7   146    0.85   0        Ruim     1.00      
-63    4.9   35     0.83   0        Ruim     0.99      
-59    2.6   7      0.42   1        Bom      0.97      
-76    5.6   56     0.69   0        Ruim     1.00      
-61    1.5   13     0.22   1        Bom      1.00      
-52    9.7   31     0.75   0        Ruim     1.00      
-83    7.9   119    0.29   0        Ruim     1.00      
-93    12.5  47     0.71   0        Ruim     1.00      
-65    2.8   26     0.70   0        Ruim     0.89      
-40    1.6   21     0.31   1        Bom      1.00      
-60    1.6   6      0.26   1        Bom      1.0

### Save

In [56]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
